In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.Collecting sklearn
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'



  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

In [6]:
customers = pd.read_csv("D:/csv file/Customers.csv")
products = pd.read_csv("D:/csv file/Products.csv")
transactions = pd.read_csv("D:/csv file/Transactions.csv")

In [7]:
# Merge datasets
merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

In [9]:
# Create a summary of customer transactions
customer_summary = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'ProductID': 'count',
    'Region': 'first'
}).reset_index()

# One-hot encoding for categorical features
customer_summary = pd.get_dummies(customer_summary, columns=['Region'], drop_first=True)

In [10]:
# Standardize the features
scaler = StandardScaler()
features = scaler.fit_transform(customer_summary[['TotalValue', 'ProductID']])

In [11]:
# Calculate similarity matrix
similarity_matrix = cosine_similarity(features)

In [13]:

def recommend_similar_customers(customer_id, top_n=3):
 
    customer_index = customer_summary[customer_summary['CustomerID'] == customer_id].index[0]
    similarity_scores = list(enumerate(similarity_matrix[customer_index]))
    sorted_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_customers = sorted_scores[1:top_n + 1]
    recommendations = []
    for index, score in top_customers:
        recommendations.append({
            'CustomerID': customer_summary.iloc[index]['CustomerID'],
            'SimilarityScore': score
        })
    
    return recommendations

In [14]:

lookalike_data = {}
for customer_id in customer_summary['CustomerID'].unique()[:20]:  
    lookalike_data[customer_id] = recommend_similar_customers(customer_id)

In [15]:

lookalike_output = {
    'CustomerID': [],
    'Lookalikes': [],
    'Scores': []
}

for cust_id, recommendations in lookalike_data.items():
    lookalike_output['CustomerID'].append(cust_id)
    lookalike_output['Lookalikes'].append([rec['CustomerID'] for rec in recommendations])
    lookalike_output['Scores'].append([rec['SimilarityScore'] for rec in recommendations])


lookalike_df = pd.DataFrame(lookalike_output)

In [16]:
# Save to CSV
lookalike_df.to_csv('Lookalike.csv', index=False)

print("Lookalike recommendations saved to Lookalike.csv")

Lookalike recommendations saved to Lookalike.csv
